In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import re
import imaplib
import email

from datetime import datetime
from dateutil.relativedelta import relativedelta
from datetime import timezone
import os
import ssl
import smtplib
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from selenium.webdriver.common.action_chains import ActionChains
from IPython.display import clear_output

In [2]:
def open_browser():
    opts = Options()
    opts.headless = False
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=opts)
    driver.maximize_window()    
    return driver

def send_email(receiver_email,subject,body,html_body):
    message = MIMEMultipart()
    message["From"] = 'truckingbot@gmail.com'
    message["To"] = receiver_email
    message["Subject"] = subject
    message["Bcc"] = receiver_email
    message.attach(MIMEText(body, "plain"))
    message.attach(MIMEText(html_body, "html"))
    text = message.as_string()
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login('truckingbot@gmail.com', 'aonopybnowvnkjpj')
        server.sendmail('truckingbot@gmail.com', receiver_email, text)
    print("Email Successfully sent to "+str(receiver_email))

def powerdatlogin(driver):
    driver.get("https://power.dat.com/login")
    print("Login page loaded. Initiating login procedure.")
    driver.find_element_by_id("username").send_keys("trieper1")
    driver.find_element_by_id("password").send_keys("Password2316")
    driver.find_element_by_id("password").send_keys(Keys.RETURN)
    print("Power.Dat Login successful!")

def redirectsearchpage(driver):
    print("Redirecting to search loads page...")
    time.sleep(2)
    driver.get("https://power.dat.com/search/loads")
    print("Redirection successful.")
    
def new_load_search(driver):
    for element in driver.find_elements_by_tag_name("button"):
        if("NEW LOAD SEARCH" in element.text):
            print(element.text)
            element.click()
            print("Click : New Load Search")
            break
            
def type_truck_input(type_list,driver):
    driver.find_element_by_id("s2id_autogen2").click()
    actions = ActionChains(driver)
    for i in range(0,5):
        actions.send_keys(Keys.BACK_SPACE).perform()
    time.sleep(1)
    print("Input trucks of type:")
    for truck_type in truck_type_list:
        print(truck_type)
        driver.find_element_by_id("s2id_autogen2").send_keys(truck_type)
        time.sleep(.5)
        driver.find_element_by_id("s2id_autogen2").send_keys(Keys.RETURN)
        time.sleep(.5)
    
def input_origin(origin,driver):
    actions = ActionChains(driver)
    origin_found=0
    for element in driver.find_elements_by_xpath("//input[@ng-model='ctrlCtx.searchCriteria.origin']"):
        if(element.get_attribute("auto-complete")=='handleOriginChange' and origin_found==0):
            element.clear()
            element.click()
            for bkspc in range(0,10):
                actions.send_keys(Keys.BACKSPACE).perform()
            element.send_keys(origin)
            print("Origin inputted:",origin)
            origin_found=1
            break
            
def truck_length_input(truck_length,driver):
    actions = ActionChains(driver)
    for element in driver.find_elements_by_tag_name("input"):
        if(element.get_attribute("ng-blur")=='validateLength(ctrlCtx.searchCriteria)'):
            element.clear()
            element.click()
            
            time.sleep(.75)
            actions.send_keys(Keys.BACK_SPACE).perform()
            actions.send_keys(Keys.BACK_SPACE).perform()
            #element.clear()
            element.send_keys(truck_length)
            time.sleep(1)
            print("Inputted Truck Length:",truck_length)
            break
    

def truck_weight_input(truck_weight,driver):
    actions = ActionChains(driver)
    for element in driver.find_elements_by_tag_name("input"):
        if(element.get_attribute("ng-blur")=='validateWeight(ctrlCtx.searchCriteria)'):
            element.clear()
            element.click()
            time.sleep(.75)
            for i in range(0,10):
                actions.send_keys(Keys.BACK_SPACE).perform()
            #actions.send_keys(Keys.BACK_SPACE).perform()
            #element.clear()
            element.send_keys(truck_weight)
            print("Inputted Truck Weight:",truck_weight)
            break           
    
    

def search_back_hours(hours,driver):
    actions = ActionChains(driver)
    for element in driver.find_elements_by_tag_name("input"):
        if(element.get_attribute("placeholder")=='hrs'):
            element.click()
            time.sleep(2)
            actions.send_keys(Keys.BACK_SPACE).perform()
            actions.send_keys(Keys.BACK_SPACE).perform()
            #element.clear()
            element.send_keys(hours)
            print("Inputted Search Back Hours:",hours)
            break           
def set_FP_parameter(driver):
    #Selecting F/P Option
    for tag in driver.find_elements_by_tag_name("option"):
        if("Both" in tag.text):
            tag.click()
            print("FP Select: Both")
            break
def submit_search(driver):
    driver.find_element_by_xpath("//button[@ng-click='submitSearch()']").click()
    print("Serching for new loads...")
    return 1

def scroll_down(driver):
    try:
        driver.find_element_by_xpath("//div[@class='filter-panel-toggle']").click()
    except:
        pass
    driver.find_element_by_id("searchResults").click()
    time.sleep(1)
    actions=ActionChains(driver)
    for i in range(0,10):
        actions.send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(.5)

def return_loads_list(driver):
    loads_list=list()
    soup=BeautifulSoup(driver.page_source,'html5lib')
    search_table=soup.find_all("table",{"class":"searchResultsTable"})[0].find_all("tr",{"class":"qa-match-row resultSummary unread"})

    for i in range(0,len(search_table)) :
        print(i)
        load_td=search_table[i]
        #if("strikeout" in str(load_td)):
        #    continue
        #print(load_td,"\n___________________________________\n")
        age=load_td.find("td",{"class":"age"}).text.strip()
        pick_up=load_td.find("td",{"class":"avail"}).text.strip()
        eq=load_td.find("td",{"class":"truck"}).text.strip()
        fp=load_td.find("td",{"class":"fp"}).text.strip()
        do=load_td.find("td",{"class":"do"}).text.strip()
        trip_origin=load_td.find("td",{"class":"origin"}).text.strip()
        trip_miles=load_td.find("td",{"class":"trip"}).text.strip()
        try:
            trip_url="https://power.dat.com/search/"+str(load_td.find("td",{"class":"trip"}).find("a")['href']).strip()
        except:
            trip_url="-"
        trip_destination=load_td.find("td",{"class":"dest"}).text.strip()
        dd=load_td.find("td",{"class":"dd"}).text.strip()
        company=load_td.find("td",{"class":["company dropdown","company dropdown open"]}).text.strip()
        try:
            matchID=re.findall(".*?MatchId=(.*?)&.*?",trip_url)[0]
            registryID=re.findall(".*?RegistryId=(.*?)&.*?",trip_url)[0]
            
            company_url=f"https://power.dat.com/search/urls?Category=CustomerDirectoryTCSIProfile&MatchId={matchID}&RegistryId={registryID}"
        except:
            company_url="-"
        contact=load_td.find("td",{"class":"contact"}).text.strip()
        length=load_td.find("td",{"class":"length"}).text.strip()
        weight=load_td.find("td",{"class":"weight"}).text.strip()
        cs=load_td.find("td",{"class":"cs"}).text.strip()
        dtp=load_td.find("td",{"class":"dtp"}).text.strip()
        rate=load_td.find("td",{"class":"rate"}).text.strip()
        book=load_td.find("td",{"class":"bookItNow"}).text.strip()
        dollar_per_mile=0
        try:
            rate_float=float(str(rate.replace("$","").replace(",","")))
            trip_miles_float=float(str(trip_miles.replace(",","")))

            dollar_per_mile=round(rate_float/trip_miles_float,3)
        except:
            print("Error in calculation dollar per mile.")

        try:
            trip_miles=float(str(trip_miles.replace(",","")))
        except:
            print("Error in converting trip miles")
            pass
        load_dict={"Age":age,"Pick_Up":pick_up,"EQ":eq,"FP":fp,"DO":do,"Origin":trip_origin,"Trip_Miles":trip_miles,
                   "TripURL":trip_url,
                   "Destination":trip_destination,"DD":dd,"Company":company,"Company_URL":company_url,
                   "Contact":contact,"Length":length,"Weight":weight,
                   "CS":cs,"Rate":rate,"Booked":book,"$PerMile":dollar_per_mile
                  }
        for key in load_dict.keys():
            print(key,":",load_dict[key])
        print("__________________________________________")
        loads_list.append(load_dict)
    return loads_list

def return_dataframe(loads_list):
    df=pd.DataFrame(loads_list)
    columns=df.columns
    df=df.sort_values(by=['$PerMile',"Trip_Miles"],ascending=False).reset_index()[columns]
    return df

# Input Parameters

In [3]:
origins=["Z1","Z2","Z3","Z4","Z5"]
truck_type_list=["D","F"]
truck_length="40"
truck_weight="10000"
hours="2"

In [ ]:
ij=1
key_order=['Rate','Trip_Miles','$PerMile','Origin','Destination','Age','Length', 'Weight', 'Contact','Pick_Up', 'EQ', 'FP', 
           'DO',  'DD', 'Company',  'CS',  'Booked','TripURL', 'Company_URL']
while(1):
    origin_count=0
    try:
        driver.quit()
    except:
        pass
    try:
        ij=0
        driver=open_browser()
        time.sleep(2)
        try:
            powerdatlogin(driver)
        except:
            print("Error in login. Trying to quit browser and try again.")
            try:
                driver.quit()
                continue
            except:
                print("Failed to quit driver.")
                continue
        for origin in origins:
            origin_count+=1
            try:
                redirectsearchpage(driver)
                time.sleep(4)
                new_load_search(driver)
            except:
                print("Error in loading new search")
                origin_count-=1
                continue


            try:
                type_truck_input(truck_type_list,driver)
            except:
                 pass
            time.sleep(1)
            input_origin(origin,driver)
            time.sleep(2)
            #set_FP_parameter(driver)
            try:
                search_back_hours(hours,driver)
                truck_weight_input(truck_weight,driver)
                truck_length_input(truck_length,driver)
                set_FP_parameter(driver)
                submit_search(driver)
            except:
                continue


            time.sleep(2)
            scroll_down(driver)
            try:
                loads_list=return_loads_list(driver)
            except:
                continue
            try:
                df=return_dataframe(loads_list)
            except:
                continue
            df['EmailTrackingID']=['' for i in range(0,df.shape[0])]
            for i in range(df.shape[0]):
                df['EmailTrackingID'][i]=str(df['Origin'][i])+str(df['Destination'][i])+str(df['Trip_Miles'][i])+str(df['Company'][i])+str(df['Pick_Up'][i])




            EmailMessage=""
            HTML_Message="<html><body>"
            sent_excel=1
            try:
                df_sent_email=pd.read_excel("SentEmail.xlsx")
                sent_details_list=list(df_sent_email['EmailTrackingID'])
            except:
                print("Sent Emails Excel not found")
                sent_excel=0
                sent_details_list=[]

            count_email=0
            #print("Reached Test 2")
            for i in range(0,df.shape[0]):
                email_track=df['EmailTrackingID'][i]
                if(email_track not in sent_details_list):
                    new_load_dict=df[i:i+1].to_dict(orient='records')[0]
                    #print(new_load_dict,"\n")
                    count_email+=1
                    HTML_Message+="<h1>"+str(count_email)+"</h1>"
                    for key in key_order:
                        if(key=="Contact"):
                            HTML_Message+="<p><h2>"+str(key)+" : "+str(new_load_dict[key])+"</h2></p>"
                            continue
                        if(key!="EmailTrackingID"):
                            HTML_Message+='<strong>'+str(key)+'</strong>'+" : "+str(new_load_dict[key])+"<br>"

                    HTML_Message+="<br>___________________________________<br>"
            HTML_Message+="</body></html>"
            if(origin_count==1):
                print("Creating df_future send.")
                df_future_send=df
                print("Length of df_future_send:",df_future_send.shape[0])
            else:
                print("Updating df_future send.")
                df_future_send=df_future_send[df.columns]
                df_future_send=pd.concat([df_future_send,df],sort=False).reset_index()
                print("Length of df_future_send:",df_future_send.shape[0])

            #print(HTML_Message)

            if(len(HTML_Message)>40):
                subject="Trucking BOT | Loads Available | ZONE : "+str(origin)
                try:
                    send_email("truckingbizsol@gmail.com",subject,EmailMessage,HTML_Message)
                    #send_email("gooddeal@coolrunningsauto.org",subject,EmailMessage,HTML_Message)
                    send_email("Jobs@coolrunningsauto.org",subject,EmailMessage,HTML_Message)
                except:
                    pass
            else:
                #print("Email Message:",HTML_Message)
                print("No new entries found.")
            #driver.quit()
            print(datetime.now())

        df_future_send.to_excel("SentEmail.xlsx")
        try:
            history_len=len(driver.find_element_by_id("searchList").find_elements_by_tag_name("tbody"))-1

            history_deletion_attempts=0
            while(history_len>4):
                try:
                    driver.find_element_by_id("searchList").find_elements_by_tag_name("tbody")[1].find_element_by_tag_name("tr").click()
                    time.sleep(1)
                    driver.find_element_by_xpath("//button[@class='delete qa-my-searches-delete']").click()
                    history_len=len(driver.find_element_by_id("searchList").find_elements_by_tag_name("tbody"))-1
                    print("History Lenth:",history_len-1)
                    time.sleep(3)
                except:
                    history_deletion_attempts+=1
                    if(history_deletion_attempts==5):
                        break
                    print("Exception in Deleting")
                    continue
        except:
            pass
        sleeping_time_minutes=5
        sleeping_time_seconds=int(sleeping_time_minutes*60)
        print(f"Sleeping for {sleeping_time_minutes} minutes","\n\n______________________________________________________________________\n\n")
        driver.quit()
        clear_output(wait=True)
        time.sleep(sleeping_time_seconds)
    except Exception as e:
        time.sleep(60)
        print("Main Exception",es)
        continue
    

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\Administrator\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


Login page loaded. Initiating login procedure.
Power.Dat Login successful!
Redirecting to search loads page...
Redirection successful.
NEW LOAD SEARCH
Click : New Load Search
Input trucks of type:
D
F
Origin inputted: Z1
Inputted Truck Weight: 10000
Inputted Truck Length: 40
FP Select: Both
Serching for new loads...
